# Notebook for building and testing the model     

In this notebook, I will build and run an initial test over the first 2,048 rows of the data to ensure proper functionality before deploying on midway2 to train on GPUs. 

In [ ]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import sys

In [ ]:
# cuda setup
device = torch.device("cuda")
kwargs = {'num_workers': 2, 'pin_memory': True} 

In [831]:
# hyper params
batch_size = 128
latent_size = 10
epochs = 10

In [ ]:
# reading in the data
dat = np.genfromtxt('traindata/trip-2021-07-28.csv', delimiter=',')[1:,]

In [ ]:
# need to subset this in a conditional fashion (i.e., each gamma value, here class c, has 1000 data points)
gamma = np.unique(dat[:,1])
idx = [np.where(dat[:,1] == gamma[i]) for i in np.arange(len(gamma))]

In [ ]:
# create an 80/20 split in each data set
temp = [train_test_split(dat[idx[i][0],0], dat[idx[i][0],2], test_size=0.2, random_state=42) for i in np.arange(len(gamma))]

# for each gamma value...
Xltrain = []
Xltest = []
altrain = []
altest = []
for t in np.arange(len(temp)):
    Xltrain.append(temp[t][0])
    Xltest.append(temp[t][1])
    altrain.append(temp[t][2])
    altest.append(temp[t][3])

In [893]:
# creating a simple matrix of training data (...x2, ...x1) 
tr_dat = TensorDataset(torch.tensor(np.vstack((np.hstack((Xltrain[0:5])), np.repeat(gamma[0:5],len(Xltrain[0])))).T), torch.tensor(np.hstack((altrain[0:5]))))

In [900]:
tr_dl = DataLoader(tr_dat, batch_size=128, shuffle=True, num_workers=2)

In [899]:
te_dat = TensorDataset(torch.tensor(np.vstack((np.hstack((Xltest[0:5])), np.repeat(gamma[0:5],len(Xltest[0])))).T), torch.tensor(np.hstack((altest[0:5]))))
te_dl = DataLoader(te_dat, num_workers=2)

In [898]:
class BaselineNet(nn.Module):
    def __init__(self, hidden1):
        super().__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(2, hidden1),
            nn.Tanh(),
            nn.Linear(hidden1, 10), 
            nn.Tanh(),
            nn.Linear(10, 1),
            nn.ReLU()
        )
        self.double()

    def forward(self, xg):
        a = self.fc1(xg)
        return a

In [901]:
bnet = BaselineNet(500)
bnet.eval()
mse_loss = nn.MSELoss()
optimizer = optim.Adagrad(bnet.parameters(), lr=2e-3)

In [878]:
len(tr_dl.dataset)

800

In [903]:
for i, (xg, a) in enumerate(tr_dl):
    print(xg)
    print(a)

    print(bnet(xg))

    if i==1:
        break

tensor([[ 1.0000e-02, -4.6416e+01],
        [ 2.0000e-03, -6.8129e+01],
        [ 2.6000e-03, -2.1544e+01],
        [ 1.1200e-02, -2.1544e+01],
        [ 9.8040e-01, -3.1623e+01],
        [ 1.9880e-01, -6.8129e+01],
        [ 3.0000e-03, -4.6416e+01],
        [ 4.2000e-03, -1.0000e+02],
        [ 8.2000e-03, -6.8129e+01],
        [ 6.2620e-01, -4.6416e+01],
        [ 6.3240e-01, -3.1623e+01],
        [ 6.1800e-02, -3.1623e+01],
        [ 1.7020e-01, -2.1544e+01],
        [ 4.0000e-04, -4.6416e+01],
        [ 9.2660e-01, -3.1623e+01],
        [ 4.4800e-02, -1.0000e+02],
        [ 3.3980e-01, -3.1623e+01],
        [ 1.3580e-01, -6.8129e+01],
        [ 5.0600e-01, -3.1623e+01],
        [ 8.6600e-02, -4.6416e+01],
        [ 6.6460e-01, -4.6416e+01],
        [ 1.7800e-02, -1.0000e+02],
        [ 2.6180e-01, -3.1623e+01],
        [ 1.8000e-03, -2.1544e+01],
        [ 1.3600e-02, -4.6416e+01],
        [ 1.9400e-02, -1.0000e+02],
        [ 5.6420e-01, -4.6416e+01],
        [ 4.9440e-01, -2.154

In [861]:
def train(dl, model, loss_fn, optim):
    size = len(dl.dataset)
    for batch, (xg, a) in enumerate(dl):
        # Compute prediction error
        pred = model(xg)
        loss = loss_fn(pred, a)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(xg)
        #     print(f"loss: {loss:>.2f}  [{current:>2d}/{size:>2d}]")

        return loss.item()

In [862]:
def test(dl, model, loss_fn):
    size = len(dl.dataset)
    num_batches = len(dl)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for xg, a in dl:
            pred = model(xg)
            test_loss += loss_fn(pred, a).item()
    test_loss /= num_batches
    return test_loss
    #print(f"Test Error: Avg loss: {test_loss:>.2f} \n")

In [902]:
epochs = 500
for t in range(epochs):
    trloss = train(tr_dl, bnet, mse_loss, optimizer)
    teloss = test(te_dl, bnet, mse_loss)
    if t%100 == 0:
        print(f"Epoch {t+1} - ")
        print(f"\tTrain loss: {trloss:.2f}")
        print(f"\tTest loss: {teloss:.2f}")
print("Done!")

Epoch 1 - 
	Train loss: 2700910.45
	Test loss: 2493463.21
Epoch 101 - 
	Train loss: 2325001.17
	Test loss: 2491017.54
Epoch 201 - 
	Train loss: 2245138.84
	Test loss: 2490569.44
Epoch 301 - 
	Train loss: 2972198.84
	Test loss: 2490225.71
Epoch 401 - 
	Train loss: 2499159.97
	Test loss: 2489936.42
Done!


In [806]:
preds = bnet(torch.tensor(np.vstack((np.hstack((Xltrain[0:5])), np.repeat(gamma[0:5],len(Xltrain[0])))).T))
preds

tensor([[5.1502],
        [5.1502],
        [5.1502],
        ...,
        [5.1500],
        [5.1500],
        [5.1500]], dtype=torch.float64, grad_fn=<AddmmBackward>)

In [ ]:
r2_score(y_true=altrain[5], y_pred=pred.detach().numpy())
#bnet(torch.unsqueeze(torch.tensor(Xltrain[5][0:1]),1), torch.unsqueeze(torch.tensor(np.repeat(gamma[5],1)),1))
#torch.unsqueeze(torch.cat((torch.tensor(Xltrain[5]),torch.tensor(Xltrain[0])),0),1)
torch.unsqueeze(torch.tensor(np.repeat(gamma[0:9],len(Xltrain[0]))),1).shape

In [ ]:
#torch.unsqueeze(torch.tensor(Xltrain[0]),1)
#bnet.forward(torch.tensor(Xltrain[0]), torch.tensor(np.repeat(gamma[0], len(Xltrain[0]))))
torch.tensor(Xltrain[0]).view(-1, torch.tensor(Xltrain[0]).size(0)).shape
torch.unsqueeze(torch.tensor(Xltrain[0]),1)
#print(torch.unsqueeze(torch.linspace(-1,1,10),-1).shape)
#bnet.forward(torch.tensor(Xltrain[0][1]),torch.tensor(gamma[0]))
#torch.unsqueeze(torch.tensor(Xltrain[0]),0).shape

## VAE implementation

Following the procedure in https://github.com/AntixK/PyTorch-VAE/blob/master/models/vanilla_vae.py, seems pretty similar to the architecture of the BaselineNet...

In [ ]:
from torch.nn import functional as F

In [ ]:
class BaseVAE(nn.Module):
    
    def __init__(self) -> None:
        super(BaseVAE, self).__init__()

    def encode(self, input: torch.Tensor) -> List[torch.Tensor]:
        raise NotImplementedError

    def decode(self, input: torch.Tensor) -> Any:
        raise NotImplementedError

    def sample(self, batch_size:int, current_device: int, **kwargs) -> torch.Tensor:
        raise RuntimeWarning()

    def generate(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        raise NotImplementedError

    @abstractmethod
    def forward(self, *inputs: torch.Tensor) -> torch.Tensor:
        pass

    @abstractmethod
    def loss_function(self, *inputs: Any, **kwargs) -> torch.Tensor:
        pass

In [881]:
# from https://debuggercafe.com/getting-started-with-variational-autoencoder-using-pytorch/
# define a simple linear VAE
class LinearVAE(nn.Module):
    def __init__(self, features):
        super(LinearVAE, self).__init__()
 
        # encoder
        self.enc1 = nn.Linear(2, 48)
        self.enc2 = nn.Linear(48, features*2)
 
        # decoder 
        self.dec1 = nn.Linear(features, 24)
        self.dec2 = nn.Linear(24, 1)

        self.double()

    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    def forward(self, x):
        # encoding
        x = F.tanh(self.enc1(x))
        x = self.enc2(x).view(-1, 2, features)

        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance

        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
 
        # decoding
        x = F.tanh(self.dec1(z))
        reconstruction = torch.sigmoid(self.dec2(x))
        return reconstruction, mu, log_var

In [ ]:
def final_loss(mse_loss, mu, logvar):
    """
    This function will add the reconstruction loss (MSELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)

    :param mse_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    MSE = mse_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE + KLD

In [896]:
batch_size = 512
model = LinearVAE(features=16) # dimensionality of the latent variable
optimizer = optim.SGD(model.parameters(), lr=2e-3)

In [888]:
def train_vae(dl, model, loss_fn, optim):
    size = len(dl.dataset)
    for batch, (xg, a) in enumerate(dl):
        # Compute prediction error
        pred, mu, logvar = model(xg)
        step_loss = loss_fn(pred, a)

        loss = final_loss(step_loss, mu, logvar)

        # Backpropagation
        optimizer.zero_grad()
        step_loss.backward()
        optimizer.step()
        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(xg)
        #     print(f"loss: {loss:>.2f}  [{current:>2d}/{size:>2d}]")

        return loss.item()

In [884]:
def test_vae(dl, model, loss_fn):
    size = len(dl.dataset)
    num_batches = len(dl)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for xg, a in dl:
            pred, _, _ = model(xg)
            test_loss += loss_fn(pred, a).item()
    test_loss /= num_batches
    return test_loss

In [897]:
epochs = 600
for t in range(epochs):
    trloss = train_vae(tr_dl, model, mse_loss, optimizer)
    teloss = test_vae(te_dl, model, mse_loss)
    if t%100 == 0:
        print(f"Epoch {t+1} - ")
        print(f"\tTrain loss: {trloss:.2f}")
        print(f"\tTest loss: {teloss:.2f}")
print("Done!")

Epoch 1 - 
	Train loss: 2877075.60
	Test loss: 2493847.20
Epoch 101 - 
	Train loss: 2341032.27
	Test loss: 2493783.15
Epoch 201 - 
	Train loss: 2051467.14
	Test loss: 2493783.09
Epoch 301 - 
	Train loss: 2134317.78
	Test loss: 2493783.07
Epoch 401 - 
	Train loss: 2318922.50
	Test loss: 2493783.05
Epoch 501 - 
	Train loss: 2566924.09
	Test loss: 2493783.04
Done!


In [763]:
model.train()
for e in range(800):
    optimizer.zero_grad()
    
    pred, mu, logvar = model(torch.tensor(np.vstack((Xltrain[0],np.repeat(gamma[0],len(Xltrain[0])))).T))
    step_loss = mse_loss(pred, torch.unsqueeze(torch.tensor(np.hstack((altrain[0]))),1))

    loss = final_loss(step_loss, mu, logvar)
    loss.backward()

    # update with current step regression parameters 
    optimizer.step()

    if e % 100 == 0:
        print ('epoch [{}], Loss: {:.2f}'.format(e, loss.item()))

epoch [0], Loss: 295960.03
epoch [100], Loss: nan
epoch [200], Loss: nan
epoch [300], Loss: nan
epoch [400], Loss: nan
epoch [500], Loss: nan
epoch [600], Loss: nan
epoch [700], Loss: nan


In [747]:
torch.tensor(np.vstack((Xltrain[0],np.repeat(gamma[0],len(Xltrain[0])))).T).shape

torch.Size([800, 2])

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n